In [3]:
import pypsa
import pandas as pd
import numpy as np

In [2]:
n_ref = pypsa.Network("elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_0export.nc")
n_1 = pypsa.Network("elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_1export.nc")
n_10 = pypsa.Network("elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_10export.nc")
n_50 = pypsa.Network("elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_50export.nc")
n_1000 = pypsa.Network("elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_1000export.nc")

INFO:pypsa.io:Imported network elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_0export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_1export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_10export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_50export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_30_ec_lc1.0_Co2L_3H_2030_0.076_AP_1000export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [33]:
def calc_util(n, tech):
    
    
    tech_index = n.generators.loc[n.generators.carrier == tech].index


    weightings = pd.DataFrame(
        np.outer(n.snapshot_weightings["generators"], [1.0] * len(tech_index)),
        index=n.snapshots,
        columns=tech_index,
    )
    
    gen_tech = (n.generators_t.p.filter(regex='{}$'.format(tech)) * weightings).sum()
    installed_cap = n.generators.p_nom_opt.filter(regex='{}$'.format(tech))
    year_avail = (n.generators_t.p_max_pu.filter(regex='{}$'.format(tech)) * weightings ).sum()
    
    max_gen_solar=(installed_cap * year_avail)
    
    return (gen_tech/max_gen_solar).mean()
    

calc_util(n_10, 'solar')    
calc_util(n_10, 'onwind')    

0.5752892741932514

In [34]:
res_techs = [
        "csp",
        "rooftop-solar",
        "solar",
        "onwind",
        "onwind2",
        "offwind",
        "offwind2",
        "ror",
    ]

def calc_additional_res(n, n_ref, tech):

    
    res_index = n.generators.filter(regex='{}$'.format(tech), axis=0).index

    res_cap = n.generators.p_nom_opt[res_index].sum()
    res_cap_ref = n_ref.generators.p_nom_opt[res_index].sum()

    return (res_cap - res_cap_ref) / 1e3

calc_additional_res(n_10, n_ref, 'onwind')

-0.008197035359922665

In [35]:
calc_additional_res(n_10, n_ref, 'solar')

10.099447628185153

In [36]:
for tech in res_techs:
    print(tech + ' : '+ str(calc_additional_res(n_1, n_ref, tech)))

csp : 4.230682896447312e-06
rooftop-solar : 1.700271989680979e-05
solar : 0.9702934932609787
onwind : 0.013841252930140399
onwind2 : 8.325940600277505e-06
offwind : 4.242823956183074e-06
offwind2 : 3.851044028500124e-06
ror : 0.0


In [37]:
for tech in res_techs:
    print(tech + ' : '+ str(calc_additional_res(n_10, n_ref, tech)))

csp : 2.1784641883463435e-08
rooftop-solar : 5.784584852341429e-06
solar : 10.099447628185153
onwind : -0.008197035359922665
onwind2 : -1.0054709760231186e-05
offwind : -4.842211148212317e-06
offwind2 : -6.899294009048082e-07
ror : 0.0


In [38]:
for tech in res_techs:
    print(tech + ' : '+ str(calc_additional_res(n_50, n_ref, tech)))

csp : -2.1477519912377965e-06
rooftop-solar : -5.947703189146114e-06
solar : 50.266332121673095
onwind : -0.09531109823949009
onwind2 : -9.889991259146156e-06
offwind : -5.0183704192519055e-06
offwind2 : -3.3231414564877313e-06
ror : 0.0


In [39]:
for tech in res_techs:
    print(tech + ' : '+ str(calc_additional_res(n_1000, n_ref, tech)))

csp : 2.419175331208022e-07
rooftop-solar : -5.888825672863335e-06
solar : 1062.1232620850017
onwind : -5.946957078893039
onwind2 : -1.3027065138763304e-05
offwind : -5.773531871247278e-06
offwind2 : -2.2176354774888634e-06
ror : 0.0


In [40]:
def calc_additional_elec(n, n_ref):

    elec_cap = n.links.filter(like='Electrolysis', axis=0).p_nom_opt.sum()
    res_cap_ref = n_ref.links.filter(like='Electrolysis', axis=0).p_nom_opt.sum()

    return (elec_cap - res_cap_ref) / 1e3


calc_additional_elec(n_1, n_ref)

0.21787638866114867

In [41]:
calc_additional_elec(n_10, n_ref)

3.9188446357426945

In [42]:
calc_additional_elec(n_50, n_ref)

20.815178573859356

In [43]:
calc_additional_elec(n_1000, n_ref)

368.42433370195243

In [44]:
for tech in res_techs:
    print('{} : {}'.format(tech, calc_util(n_1, tech)))

csp : 0.5605478741990101
rooftop-solar : 0.6481263744924569
solar : 0.9513813653653801
onwind : 0.6121799281359066
onwind2 : 0.5128333104774357
offwind : 0.5526045893231663
offwind2 : 0.556848043388617
ror : inf


In [45]:
for tech in res_techs:
    print('{} : {}'.format(tech, calc_util(n_10, tech)))

csp : 0.5169846505638388
rooftop-solar : 0.6185562016453998
solar : 0.9442287758620823
onwind : 0.5752892741932514
onwind2 : 0.4615331295983346
offwind : 0.4741897065755268
offwind2 : 0.5146707061844739
ror : inf


In [46]:
for tech in res_techs:
    print('{} : {}'.format(tech, calc_util(n_50, tech)))

csp : 0.5579265184313577
rooftop-solar : 0.6532331543024901
solar : 0.95119559979548
onwind : 0.5976399736333219
onwind2 : 0.48116287775190103
offwind : 0.521003500414976
offwind2 : 0.5554607954976702
ror : inf


In [47]:
for tech in res_techs:
    print('{} : {}'.format(tech, calc_util(n_1000, tech)))

csp : 0.5704702034901293
rooftop-solar : 0.6111801316635725
solar : 0.9481614661079637
onwind : 0.6006551779019075
onwind2 : 0.46738038376480495
offwind : 0.4985468476541269
offwind2 : 0.5322104958618579
ror : inf


In [56]:
def calc_elec_capa(n):

    elec_cap = n.links.filter(like='Electrolysis', axis=0).p_nom_opt.sum() *8760
    elec_output = n.links_t.p0.filter(like='Electrolysis').sum().sum() * 3

    return elec_output / elec_cap
calc_elec_capa(n_1)  

0.4169245649539059

In [57]:
calc_elec_capa(n_10)  

0.4138362439177809

In [58]:
calc_elec_capa(n_50)  

0.4061398074025122

In [59]:
calc_elec_capa(n_1000)  

0.3825493675839524

In [69]:
def calc_elec_capa_exp(n):

    export_buses = n.links.loc[n.links.bus1 == 'H2 export bus', 'bus0'] + ' Electrolysis'

    elec_cap = n.links.loc[export_buses].p_nom_opt.sum() *8760
    elec_output = n.links_t.p0[export_buses].sum().sum() * 3

    return elec_output / elec_cap
calc_elec_capa_exp(n_1)  

0.4689551637830714

In [70]:
calc_elec_capa_exp(n_10)  

0.41331740711677944

In [71]:
calc_elec_capa_exp(n_1000)  

0.38098796155593445

In [11]:
import geopandas as gpd
df = pd.read_csv("/home/raj08555/dev/sandbox-cs/pypsa-earth-sec/data/export_ports.csv", keep_default_na=False,
        na_values=[""])
geometry = gpd.points_from_xy(df.y, df.x)
geo_df = gpd.GeoDataFrame(df, #specify our data
                        crs='epsg:4326', #specify our coordinate reference system
                        geometry=geometry) #specify the geometry list we created   
#geo_df

geo_df = geo_df[geo_df["country"=="NA"]]

KeyError: False